In [176]:
from __future__ import division, print_function 
import os
import sys
import numpy as np
import pandas as pd
import itertools as it
#import math
from matplotlib import pyplot as plt
import seaborn as sns
import copy

In [177]:
# example about how to pair n elements in a list
people = ["Lisa","Pam","Maurice","Richard","John","Graham"]

def tessa(source):
        result = []
        for p1 in range(len(source)):
                for p2 in range(p1+1,len(source)):
                        result.append([source[p1],source[p2]])
        return result

pairings = tessa(people)
print("%d pairings" % len(pairings))
for pair in pairings:
        print(pair)

15 pairings
['Lisa', 'Pam']
['Lisa', 'Maurice']
['Lisa', 'Richard']
['Lisa', 'John']
['Lisa', 'Graham']
['Pam', 'Maurice']
['Pam', 'Richard']
['Pam', 'John']
['Pam', 'Graham']
['Maurice', 'Richard']
['Maurice', 'John']
['Maurice', 'Graham']
['Richard', 'John']
['Richard', 'Graham']
['John', 'Graham']


## This implementation is just for ONE issue over 12 dimensions


## ============== First Part ===============
### The following codes are divided into several parts to calculate the agreement level among all experts

### 1. As every experts have picked a score for the issue, based on the issue, calculate the preference matrix of all pair of scores for each expert. 
This is done by comparing the distance of the two scores to the picked one. That is: 
##### `p(n,m) = 0.5 - (dn - dm)`, 
where p > 0.5 means prefer n to m, d means `distance = 0.1 * fabs (score_i - score_picked)`

In [178]:
score_range = range(6)
exp_scores = [1,2,5,5,5,5,5,4,4,4] 

In [179]:
# after the five star step, I SHOULD get the database about the chosen score for the issue of every experts
pre_list = []
for exp in range(len(exp_scores)):
    P = np.eye(6) # preference matrix
    for n in score_range:
        for m in score_range:
            #print(n,m)
            d_n = np.fabs(0.1 * (n - exp_scores[exp]))
            d_m = np.fabs(0.1 * (m - exp_scores[exp]))
            P[n,m] = 0.5 - (d_n - d_m)
            P[m,m] = np.nan #compare the score with itself is meaningless
    #print(P)
    pre_list.append(P)
print(pre_list)

[array([[ nan,  0.4,  0.5,  0.6,  0.7,  0.8],
       [ 0.6,  nan,  0.6,  0.7,  0.8,  0.9],
       [ 0.5,  0.4,  nan,  0.6,  0.7,  0.8],
       [ 0.4,  0.3,  0.4,  nan,  0.6,  0.7],
       [ 0.3,  0.2,  0.3,  0.4,  nan,  0.6],
       [ 0.2,  0.1,  0.2,  0.3,  0.4,  nan]]), array([[ nan,  0.4,  0.3,  0.4,  0.5,  0.6],
       [ 0.6,  nan,  0.4,  0.5,  0.6,  0.7],
       [ 0.7,  0.6,  nan,  0.6,  0.7,  0.8],
       [ 0.6,  0.5,  0.4,  nan,  0.6,  0.7],
       [ 0.5,  0.4,  0.3,  0.4,  nan,  0.6],
       [ 0.4,  0.3,  0.2,  0.3,  0.4,  nan]]), array([[ nan,  0.4,  0.3,  0.2,  0.1,  0. ],
       [ 0.6,  nan,  0.4,  0.3,  0.2,  0.1],
       [ 0.7,  0.6,  nan,  0.4,  0.3,  0.2],
       [ 0.8,  0.7,  0.6,  nan,  0.4,  0.3],
       [ 0.9,  0.8,  0.7,  0.6,  nan,  0.4],
       [ 1. ,  0.9,  0.8,  0.7,  0.6,  nan]]), array([[ nan,  0.4,  0.3,  0.2,  0.1,  0. ],
       [ 0.6,  nan,  0.4,  0.3,  0.2,  0.1],
       [ 0.7,  0.6,  nan,  0.4,  0.3,  0.2],
       [ 0.8,  0.7,  0.6,  nan,  0.4,  0.3],
   

### ======== Problem =========
### Should I rewrite the code by 'dataFrame' to make the preference array more clear to consult in later step?

### 2. For every pair of experts ek and eb, evaluate their similarity by calculating the similarity matrix. 
#### ======    `SM_kb(i,j) = 1 - |p_k(i,j) - p_b(i,j)|`       ======
This means for EVERY same pair of scores (eg. 1&4), the two experts MAY have different preference (p_k[1,4] & p_b[1,4]), then their similarity is: 
#### `SM_kb[1,4] = 1 - |p_k(1,4) - p_b(1,4)| `
Note that because `p_k(i,j) = 1 - p_k(j,i)`, we can know that `|p_k(i,j)-p_b(i,j)|`= `|1-p_k(j,i)-1 + p_b(j,i)|` = `|p_b(j,i)-p_k(j,i)|` = `|p_k(j,i)-p_b(j,i)|`. Therefore `SM_kb(i,j)=SM_kb(j,i)`.

In [180]:
#use the code above to generate the similarity matrixs
pairs = tessa(pre_list)
Sim_list = []
for pair in pairs:
    #print(pair[0])
    # SM_kb(i,j) = 1 - |p_k(i, j) - p_b(i, j)|
    Sim = 1 - np.fabs(pair[0]-pair[1])
    #print(Sim)
    Sim_list.append(Sim) # a list of all pair of experts' similarity matrixs
    
print(Sim_list)

[array([[ nan,  1. ,  0.8,  0.8,  0.8,  0.8],
       [ 1. ,  nan,  0.8,  0.8,  0.8,  0.8],
       [ 0.8,  0.8,  nan,  1. ,  1. ,  1. ],
       [ 0.8,  0.8,  1. ,  nan,  1. ,  1. ],
       [ 0.8,  0.8,  1. ,  1. ,  nan,  1. ],
       [ 0.8,  0.8,  1. ,  1. ,  1. ,  nan]]), array([[ nan,  1. ,  0.8,  0.6,  0.4,  0.2],
       [ 1. ,  nan,  0.8,  0.6,  0.4,  0.2],
       [ 0.8,  0.8,  nan,  0.8,  0.6,  0.4],
       [ 0.6,  0.6,  0.8,  nan,  0.8,  0.6],
       [ 0.4,  0.4,  0.6,  0.8,  nan,  0.8],
       [ 0.2,  0.2,  0.4,  0.6,  0.8,  nan]]), array([[ nan,  1. ,  0.8,  0.6,  0.4,  0.2],
       [ 1. ,  nan,  0.8,  0.6,  0.4,  0.2],
       [ 0.8,  0.8,  nan,  0.8,  0.6,  0.4],
       [ 0.6,  0.6,  0.8,  nan,  0.8,  0.6],
       [ 0.4,  0.4,  0.6,  0.8,  nan,  0.8],
       [ 0.2,  0.2,  0.4,  0.6,  0.8,  nan]]), array([[ nan,  1. ,  0.8,  0.6,  0.4,  0.2],
       [ 1. ,  nan,  0.8,  0.6,  0.4,  0.2],
       [ 0.8,  0.8,  nan,  0.8,  0.6,  0.4],
       [ 0.6,  0.6,  0.8,  nan,  0.8,  0.6],
   

### 3. evaluate the agreement level
#### There are 3 steps of this procedure:
* `consensus matrix (pair of alternatives)`: `cp[i,j]= mean(sm_12[i,j], ..., sm_1n[i,j], sm_23[i,j],..., sm_2n[i,j],..., sm_(n-1)n[i,j])`
* `each alternatives`: `ca[i] = mean(cm[i,1], ..., cm[i,n])`
* `overall agreement`: `cr = mean(ca[i],...,ca[n])`

#### ========= pair consensus level ==========
For every 'pair' of scores, the consensus level of them is the average of similarity over each similarity matrixs, i.e. the comparation of all pair of experts

In [181]:
# consensus level of pairs 
cp = sum(Sim_list)/len(Sim_list)
cp

array([[        nan,  1.        ,  0.96      ,  0.88888889,  0.81777778,
         0.70666667],
       [ 1.        ,         nan,  0.96      ,  0.88888889,  0.81777778,
         0.70666667],
       [ 0.96      ,  0.96      ,         nan,  0.92888889,  0.85777778,
         0.74666667],
       [ 0.88888889,  0.88888889,  0.92888889,         nan,  0.92888889,
         0.81777778],
       [ 0.81777778,  0.81777778,  0.85777778,  0.92888889,         nan,
         0.88888889],
       [ 0.70666667,  0.70666667,  0.74666667,  0.81777778,  0.88888889,
                nan]])

#### ======== alternatives ========
Turn the consensus level of pairs of score to each single score. Find all pairs of scores that contains the score `l` in 'pair consensus' matrix (last step), then take the average of them. 

##### 2. When the 'eye' of the array is 'Nan'
Still caculate the mean of pairs containing the score i by just ignore the 'Nan' value. First turn 'Nan' to 0, then reduce the length of the array by 1. 

In [182]:
ca = np.nansum(cp, axis=1)/5
ca

array([ 0.87466667,  0.87466667,  0.89066667,  0.89066667,  0.86222222,
        0.77333333])

#### ======== over all agreement ========
Simply average all level of the alternatives (cp)

In [183]:
cr = np.mean(ca)
cr

0.86103703703703705

In [184]:
if cr < 0.85:
    for c in ca:
        if c < cr:
            print(c)

In [185]:
np.mean(exp_scores)

4.0

## ==================Second Part ===================

#### This part is to generate the recommendations if the agreement level is below threshold 0.85
The basic idea is that, find the experts whose opinions deviate most from the majority, then suggest them to lower or uppper the scores depending on the deviation dirction. 

#### ========= collective preference ========
represents the average opinion over all experts for each pair of scores. 

In [186]:
coll = sum(pre_list)/len(pre_list)
coll

array([[  nan,  0.4 ,  0.32,  0.26,  0.2 ,  0.2 ],
       [ 0.6 ,   nan,  0.42,  0.36,  0.3 ,  0.3 ],
       [ 0.68,  0.58,   nan,  0.44,  0.38,  0.38],
       [ 0.74,  0.64,  0.56,   nan,  0.44,  0.44],
       [ 0.8 ,  0.7 ,  0.62,  0.56,   nan,  0.5 ],
       [ 0.8 ,  0.7 ,  0.62,  0.56,  0.5 ,   nan]])

#### ======== proximity values ========
For each pair of scores `l & k`, how much does your opinion deviate from the overall opinion (collective preference)? For each expert `i`, his/her preference relation with collective preference pc is:
#### `pv_i[l,k] = 1- |p_i[l,k]-coll[l,k]|` 
So the higher pv, the cloer to the average opinion

In [187]:
pv_list = []
for prefer in pre_list:
    pv = 1 - np.fabs(prefer - coll)
    pv_list.append(pv)
pv_list = np.array(pv_list)
print (pv_list)

[[[  nan  1.    0.82  0.66  0.5   0.4 ]
  [ 1.     nan  0.82  0.66  0.5   0.4 ]
  [ 0.82  0.82   nan  0.84  0.68  0.58]
  [ 0.66  0.66  0.84   nan  0.84  0.74]
  [ 0.5   0.5   0.68  0.84   nan  0.9 ]
  [ 0.4   0.4   0.58  0.74  0.9    nan]]

 [[  nan  1.    0.98  0.86  0.7   0.6 ]
  [ 1.     nan  0.98  0.86  0.7   0.6 ]
  [ 0.98  0.98   nan  0.84  0.68  0.58]
  [ 0.86  0.86  0.84   nan  0.84  0.74]
  [ 0.7   0.7   0.68  0.84   nan  0.9 ]
  [ 0.6   0.6   0.58  0.74  0.9    nan]]

 [[  nan  1.    0.98  0.94  0.9   0.8 ]
  [ 1.     nan  0.98  0.94  0.9   0.8 ]
  [ 0.98  0.98   nan  0.96  0.92  0.82]
  [ 0.94  0.94  0.96   nan  0.96  0.86]
  [ 0.9   0.9   0.92  0.96   nan  0.9 ]
  [ 0.8   0.8   0.82  0.86  0.9    nan]]

 [[  nan  1.    0.98  0.94  0.9   0.8 ]
  [ 1.     nan  0.98  0.94  0.9   0.8 ]
  [ 0.98  0.98   nan  0.96  0.92  0.82]
  [ 0.94  0.94  0.96   nan  0.96  0.86]
  [ 0.9   0.9   0.92  0.96   nan  0.9 ]
  [ 0.8   0.8   0.82  0.86  0.9    nan]]

 [[  nan  1.    0.98  0.94  0.9 

#### ========= what should change =========
The pair of scores whose 'pair consensus level' (cp) plus itself consensus level(ca) that are not enough are identified to be changed.

In [188]:
score1 = np.nanargmin(ca)
score2 = np.nanargmin(cp[score1,:])

print(score1, score2)

5 0


#### ======== who should change =========
We now have the pairs that need to be changed. For those pairs, the expert `e(i)` whose preference on them is furthest from collective preference should change the opinion. An average proximity is calcualted to identify the experts. 
##### `mean(pv_1[l,k], pv_2[l,k], ... , pv_n[l,k])`

In [189]:
def nlargest(arr, size):
    i = 0
    nl = []
    while i < size:
        nl.append(sorted(arr)[i])
        i=i+1
    return(nl)

In [190]:
avg = pv_list[:, score1, score2].mean()
print(avg)
deviateExps = [] 
for exp in range(len(exp_scores)):
    if pv_list[exp, score1, score2] < avg:
        deviateExps.append(exp)
#print(deviateExps)
if len(deviateExps) > 0.4*len(exp_scores):
    max_ch = int(round(0.4*len(exp_scores)))
    dvScores = []
    for ep in deviateExps:
        dvScores.append(exp_scores[ep])

    dvExps = set()
    t = 0
    for ep in deviateExps:
        while t < max_ch:
            if exp_scores[ep] in nlargest(dvScores, max_ch):
                dvExps.add(ep)
            t = t + 1
    deviateExps = dvExps
    print(dvExps)
    
    

0.8


In [191]:
max_ch

4

#### the suggested experts, compare their similarity and the unsuggested experts. 

In [192]:
aggreScores = [x for ep,x in enumerate(exp_scores) if ep not in deviateExps]
agg_avg = sum(aggreScores)/len(aggreScores)
print(agg_avg)

for ep in deviateExps:
    if exp_scores[ep] < agg_avg:
        print("expert{0}, your opinion"
          " is different from others, would you accept raising your score to {1}?"
          .format(ep,int(round(agg_avg))))
    else:
        print("expert{0}, your opinion"
          " is different from others, would you accept lowering your score to {1}?"
          .format(ep,int(round(agg_avg))))
print(aggreScores)


4.625
expert0, your opinion is different from others, would you accept raising your score to 5?
expert1, your opinion is different from others, would you accept raising your score to 5?
[5, 5, 5, 5, 5, 4, 4, 4]
